<a href="https://colab.research.google.com/github/Jhaveth/Simulacion-ll/blob/main/Practica_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **PRUEBAS**

In [ ]:
import math
from scipy.stats import norm
from scipy.stats import chi2
import os, sys
import random

## **Datos y Nivel de confianza**

In [ ]:
datos =[random.random() for _ in range(100)]
Nivel_confianza=95

## PRUEBA DE MEDIAS

In [ ]:
def media(X):
  return  sum(X)/len(X)
def z(alpha):
  alpha_limite = 1 - alpha + alpha/2
  z= norm.ppf(alpha_limite, loc=0, scale=1)
  return z
  
def limite_inferior_me(alpha,n):
   return 0.5 - z(alpha) *(1/math.sqrt(12*n))
def limite_superior_me(alpha,n):
   return 0.5 + z(alpha) *(1/math.sqrt(12*n))
def prueba_media(alpha,X):
  me_x = media(X)
  li_me = limite_inferior_me(alpha, len(X))
  print(f"Limite inferior de la media: {li_me} ")
  ls_me = limite_superior_me(alpha, len(X))
  print(f"Limite superior de la media: {ls_me} ")
  if li_me <= me_x <= ls_me:
    print(f"El valor de la media = {me_x} ")
    print(f"Se encuentra en los limites de aceptación")
  else:
    print(f"El valor de la media = {me_x}")
    print(f"No se encuentra en los limites de aceptación")

if __name__=="__main__":
  X=datos
  alpha= 1 - (Nivel_confianza/100)
prueba_media(alpha,X)

Limite inferior de la media: 0.4434207132961914 
Limite superior de la media: 0.5565792867038086 
El valor de la media = 0.5183850199048092 
Se encuentra en los limites de aceptación


## PRUEBA DE VARIANZA

In [ ]:
def varianza(X):
  me=media(X)
  acumulado=0
  for x in X:
     acumulado += (x-me)**2
  return acumulado / (len(X)-1)

def limite_inferior_varianza(alpha, n):
  chi_cuadrada=chi2.ppf(alpha/2, n-1)
  return chi_cuadrada/(12*(n-1))

def limite_superior_varianza(alpha, n):
  chi_cuadrada=chi2.ppf(1-(alpha/2), n-1)
  return chi_cuadrada/(12*(n-1))

def prueba_varianza(alpha,X):
  var_x=varianza(X)
  li_varianza=limite_inferior_varianza(alpha, len(X))
  print(f"Limite inferior de la varianza: {li_varianza} ")
  ls_varianza=limite_superior_varianza(alpha, len(X))
  print(f"Limite superior de la varianza: {ls_varianza} ")
  if li_varianza <= var_x <= ls_varianza:
    print(f"El valor de la varianza = {var_x} ")
    print(f"Se encuentra en los limites de aceptación")
  else:
    print(f"El valor de la varianza = {var_x}")
    print(f"No se encuentra en los limites de aceptación")


prueba_varianza(alpha,X)

Limite inferior de la varianza: 0.06175175100276404 
Limite superior de la varianza: 0.10809931704026962 
El valor de la varianza = 0.09106291613069491 
Se encuentra en los limites de aceptación


## PRUEBA JI-CUADRADA

In [ ]:
class  ChiCuadrado ( object ):
  def __init__(self, valores):
        self.valores = valores
        self.n = valores.__len__()
        self.m = math.sqrt(self.n)
        self.rango_menor = list({})
        self.rango_mayor = list({})
        self.fo = list({})
        self.cuadrados = list({})

        self.sum_fo = 0
        self.sum_cuadrados = 0

        self.fe = int((self.n) / math.floor( math.sqrt(self.n)))

  def start(self):
        paso = round(1.0 / self.fe,2)
        for i in range(self.fe):
            if i == 0 :
                self.rango_menor.append(0)
            else:
                self.rango_menor.append(self.rango_mayor[i-1])
            self.rango_mayor.append( self.rango_menor[i] + paso )
        self.rango_mayor[i] = 1.0 
        self.iniciar_conteo()

  def iniciar_conteo(self):
        self.sum_fo = 0
        self.sum_cuadrados = 0
        for i in range(self.fe):
            if i != (self.fe - 1):
                self.fo.append( self.contar( self.rango_menor[i], self.rango_mayor[i], ">" ) )
            else:
                self.fo.append( self.contar( self.rango_menor[i], self.rango_mayor[i], "]" ) )
            self.cuadrados.append(math.pow(self.fe - self.fo[i],2)/self.fe)
            self.sum_fo += self.fo[i]
            self.sum_cuadrados += self.cuadrados[i]

  def contar(self, rango_inferior, rango_superior, llave_derecha):
        contador = 0
        for i in range(self.n):
            if llave_derecha == ">":
                if ( rango_inferior <= valores[i] ) and ( valores[i] < rango_superior ):
                    contador += 1
            elif llave_derecha == "]":
                if ( rango_inferior <= valores[i] ) and ( valores[i] <= rango_superior ):
                    contador += 1
        return contador

  def reporte(self):
        str = ' {0:2s}| {1:14s}  |  {2:3s}|  {3:3s}|        |((FE-FO^2)/FE|'
        print (str.format('i','rangos','FO','FE'))
        for i in range(self.fe):
            if i != (self.fe - 1 ) :
                str = '{0:2d} | [ {1:1.2f} : {2:1.2f} > | {3:3d} | {4:3d} | {5:-6.2f}/{4:d} | {7:-3.2f}'
            else :
                str = '{0:2d} | [ {1:1.2f} : {2:1.2f} ] | {3:3d} | {4:3d} | {5:-6.2f}/{6:d} | {7:-3.2f}'
            print (str.format(i+1,
                                self.rango_menor[i],
                                self.rango_mayor[i],
                                self.fo[i],
                                self.fe,
                                math.pow(self.fo[i]-self.fe,2),self.fe,
                                self.cuadrados[i]))
        str = '{0:-26d} |{1:4d} | {2:-16.2f}'
        print (str.format(self.sum_fo, int(self.m)*self.fe, self.sum_cuadrados))


if __name__ == "__main__":
    f =X
    valores = f

    test = ChiCuadrado(valores)
    test.start()
    
    print (test.reporte())

 i | rangos          |  FO |  FE |        |((FE-FO^2)/FE|
 1 | [ 0.00 : 0.10 > |  15 |  10 |  25.00/10 | 2.50
 2 | [ 0.10 : 0.20 > |   8 |  10 |   4.00/10 | 0.40
 3 | [ 0.20 : 0.30 > |   3 |  10 |  49.00/10 | 4.90
 4 | [ 0.30 : 0.40 > |   6 |  10 |  16.00/10 | 1.60
 5 | [ 0.40 : 0.50 > |  14 |  10 |  16.00/10 | 1.60
 6 | [ 0.50 : 0.60 > |  13 |  10 |   9.00/10 | 0.90
 7 | [ 0.60 : 0.70 > |   7 |  10 |   9.00/10 | 0.90
 8 | [ 0.70 : 0.80 > |   9 |  10 |   1.00/10 | 0.10
 9 | [ 0.80 : 0.90 > |  15 |  10 |  25.00/10 | 2.50
10 | [ 0.90 : 1.00 ] |  10 |  10 |   0.00/10 | 0.00
                       100 | 100 |            15.40
None


## PRUEBA KOLMOGOROV-SMIRNOV

In [ ]:
class KolgomorovSmirnov(object):
   def __init__(self, numeros):
        self.numeros = numeros  # cada valor r[i]
        self.n = self.numeros.__len__()
        self.d_mas   = -1
        self.d_menos = -1
        self.d       = -1
        self.start()
   def ordenar_valores(self):
        import sys,os
        self.numeros
      
   def start(self):
        self.ordenar_valores()
        self.d_mas = self.calcular_d_mas()
        self.d_menos = self.calcular_d_menos()
        self.d = max(self.d_mas, self.d_menos)

   def calcular_d_mas(self):
        lista = list({})
        for i in range(0, self.n): # desde 0 hasta n-1           
            lista.append(((i+1.0)/self.n)-self.numeros[i])
            print ("%5.2f - %5.2f" % (((i+1.0)/self.n), self.numeros[i]))
        
        return max(lista)

   def calcular_d_menos(self):
        lista = list({})
        for i in range(0, self.n): # desdsde 0 hasta n-1
            print ("%5.2f - %5.2f"%( self.numeros[i],((i*1.0)/self.n)))
            lista.append( self.numeros[i]*1.0-((i*1.0)/self.n) ) # el algoritmo dice i-1 pero mi i comienza en 0
        
        return max(lista)

   def __str__(self):
        return "n : {0:-5d}\nD+: {1:-5.2f}\nD-: {2:-5.2f}\nD : {3:-5.2f}".format(self.n,self.d_mas,self.d_menos,self.d)
if __name__ == '__main__':
    import sys,os
    valores = X
    ks = KolgomorovSmirnov(valores)
    
    print (ks)

 0.01 -  0.94
 0.02 -  0.02
 0.03 -  0.87
 0.04 -  0.28
 0.05 -  0.02
 0.06 -  0.38
 0.07 -  0.57
 0.08 -  0.41
 0.09 -  0.06
 0.10 -  0.72
 0.11 -  0.08
 0.12 -  0.87
 0.13 -  0.93
 0.14 -  0.68
 0.15 -  0.97
 0.16 -  0.42
 0.17 -  0.38
 0.18 -  0.00
 0.19 -  0.86
 0.20 -  0.48
 0.21 -  0.48
 0.22 -  0.47
 0.23 -  0.80
 0.24 -  0.43
 0.25 -  0.62
 0.26 -  0.92
 0.27 -  0.57
 0.28 -  0.40
 0.29 -  0.09
 0.30 -  0.70
 0.31 -  0.47
 0.32 -  0.83
 0.33 -  0.95
 0.34 -  0.59
 0.35 -  0.22
 0.36 -  0.67
 0.37 -  0.09
 0.38 -  0.52
 0.39 -  0.34
 0.40 -  0.05
 0.41 -  0.83
 0.42 -  0.72
 0.43 -  0.88
 0.44 -  0.44
 0.45 -  0.51
 0.46 -  0.54
 0.47 -  0.99
 0.48 -  0.50
 0.49 -  0.15
 0.50 -  0.06
 0.51 -  0.56
 0.52 -  0.27
 0.53 -  0.91
 0.54 -  0.53
 0.55 -  0.11
 0.56 -  0.88
 0.57 -  0.39
 0.58 -  0.64
 0.59 -  0.10
 0.60 -  0.13
 0.61 -  0.71
 0.62 -  0.76
 0.63 -  0.44
 0.64 -  0.75
 0.65 -  0.81
 0.66 -  0.67
 0.67 -  0.11
 0.68 -  0.07
 0.69 -  0.98
 0.70 -  0.07
 0.71 -  0.87
 0.72 